<a href="https://colab.research.google.com/github/LuisaPolicarpo/Project3/blob/main/Optimization_project3_geral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## With the tables needed for the model

In [ ]:
### TITLE RATINGS & TITLE EPISODE 

## TITLE RATINGS - Contains the IMDb rating and votes information for titles.
# tconst (string) - alphanumeric unique identifier of the title
# averageRating – weighted average of all the individual user ratings
# numVotes - number of votes the title has received

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import numpy as np
import datetime

from PIL import Image 

import pickle

In [2]:
col_list = ['tconst','primaryTitle', 'startYear','genres','titleType']
title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', compression = 'gzip', sep = '\t', usecols=col_list)

In [3]:
col_list2 = ['tconst','averageRating','numVotes']
title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', compression = 'gzip', sep = '\t', usecols=col_list2)

In [4]:
t_b_r = pd.merge(title_basics, title_ratings, how='inner', on=['tconst', 'tconst'])

In [5]:
t_b_r.head()

,tconst,titleType,primaryTitle,startYear,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,1894,"Documentary,Short",5.7,1922
1,tt0000002,short,Le clown et ses chiens,1892,"Animation,Short",5.8,259
2,tt0000003,short,Pauvre Pierrot,1892,"Animation,Comedy,Romance",6.5,1735
3,tt0000004,short,Un bon bock,1892,"Animation,Short",5.6,174
4,tt0000005,short,Blacksmith Scene,1893,"Comedy,Short",6.2,2548


In [6]:
# Tables with just movies and shorts with the start year of 2022 or older.

condition = t_b_r['titleType'] == 'movie'
condition2 = t_b_r['titleType'] == 'short'
condition7 = t_b_r['startYear'] <= '2022'
m_s = t_b_r[condition | condition2 & condition7 ][['tconst','titleType','primaryTitle','startYear','genres','averageRating','numVotes']]

In [7]:
m_s.to_pickle('/content/tmg.pickle')

In [8]:
j_c = pd.read_pickle("/content/tmg.pickle")

In [9]:
j_c.head(9)

,tconst,titleType,primaryTitle,startYear,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,1894,"Documentary,Short",5.7,1922
1,tt0000002,short,Le clown et ses chiens,1892,"Animation,Short",5.8,259
2,tt0000003,short,Pauvre Pierrot,1892,"Animation,Comedy,Romance",6.5,1735
3,tt0000004,short,Un bon bock,1892,"Animation,Short",5.6,174
4,tt0000005,short,Blacksmith Scene,1893,"Comedy,Short",6.2,2548
5,tt0000006,short,Chinese Opium Den,1894,Short,5.1,175
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,1894,"Short,Sport",5.4,797
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,1894,"Documentary,Short",5.4,2064
8,tt0000009,movie,Miss Jerry,1894,Romance,5.3,200


In [11]:
# add the wheighted_IMDB column ( we took the formula from the imdb)

C = j_c['averageRating'].mean()
j_c['wheighted_IMDB'] = ((j_c['numVotes']/(j_c['numVotes']+25000))*j_c['averageRating']) + ((25000/(j_c['numVotes']+25000))*C)

In [ ]:
# Split and explode so we have just one genre per row

j_c["genres"] = j_c.genres.str.split(',')
j_c_split = j_c.explode("genres")

In [13]:
j_c_split.to_pickle('/content/movies_imdb.pickle')